In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install biopython

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from Bio.SeqIO import parse
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from sklearn.preprocessing import LabelEncoder
import re
import keras
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from collections import Counter
from sklearn.utils import class_weight
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
df = pd.read_csv('/content/drive/My Drive/USA_removed.csv')

In [ ]:
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))

LabelEncoder()

In [ ]:
def ordinal_encoder(my_array):
    
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, z
    
    return float_encoded

In [ ]:
data=df[["Sequence","Geo_Location"]]
data=data[data["Sequence"].notna()]

dummy=[]
dum=np.array(dummy)

form={"inp_seq":dum}

seq_df = pd.DataFrame (form, columns = ['inp_seq'])

seq_list=[]

for idx, seq in enumerate(list(data["Sequence"])):
    arr=ordinal_encoder(string_to_array(seq))
    seq_list.append(arr)
    
seq_df["inp_seq"]=seq_list

final_data= data.assign(enc_seq=seq_df)

final_data=final_data[["enc_seq","Geo_Location"]]

final_data

,enc_seq,Geo_Location
0,"[0.5, 0.5, 0.25, 0.25, 0.5, 1.0, 1.0, 1.0, 0.5...",Virginia
1,"[1.0, 0.5, 0.5, 0.5, 0.25, 0.75, 0.75, 1.0, 0....",Virginia
2,"[0.25, 0.5, 0.5, 0.25, 0.25, 0.5, 0.5, 0.25, 0...",Virginia
3,"[0.25, 0.5, 1.0, 1.0, 1.0, 0.5, 0.75, 0.25, 1....",Virginia
4,"[0.25, 0.25, 0.25, 0.75, 0.75, 1.0, 1.0, 1.0, ...",Virginia
...,...,...
4479,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",California
4480,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Washington
4481,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Washington
4482,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",California


In [ ]:
def get_maxLen(enc_seq):
    
    max=0
    for row in enc_seq:
        #print(type(row))
        if(len(row)>max):
            max=len(row)
    
    
    return max

In [ ]:
def append_arr(enc_seq, max_len):
    
    seq_l=list(enc_seq)
    for i in range(len(seq_l),max_len):
        seq_l.append(0)
        
    new_seq_ar=np.array(seq_l)

        
    return new_seq_ar

In [ ]:
import math

max_len=get_maxLen(final_data["enc_seq"])
print("max_len is",max_len)
if max_len%2 !=0:
    max_len += 1

n = 1
for i in range(1, math.floor(math.sqrt(max_len))):
    if max_len%i==0:
        n = i

padded_seq_list=[]

for index, row in final_data.iterrows():
    seq_ar=append_arr(row["enc_seq"],max_len)
    padded_seq_list.append(seq_ar)

dummy=[]
dum=np.array(dummy)
form={"padded_enc_seq":dum}
padded_seq_df = pd.DataFrame (form, columns = ['padded_enc_seq'])

padded_seq_df["padded_enc_seq"]=padded_seq_list

padded_final_data= final_data.assign(padded_enc_seq=padded_seq_df)

padded_final_data=padded_final_data[["padded_enc_seq","Geo_Location"]]

padded_final_data.head(5)

max_len is 29921


,padded_enc_seq,Geo_Location
0,"[0.5, 0.5, 0.25, 0.25, 0.5, 1.0, 1.0, 1.0, 0.5...",Virginia
1,"[1.0, 0.5, 0.5, 0.5, 0.25, 0.75, 0.75, 1.0, 0....",Virginia
2,"[0.25, 0.5, 0.5, 0.25, 0.25, 0.5, 0.5, 0.25, 0...",Virginia
3,"[0.25, 0.5, 1.0, 1.0, 1.0, 0.5, 0.75, 0.25, 1....",Virginia
4,"[0.25, 0.25, 0.25, 0.75, 0.75, 1.0, 1.0, 1.0, ...",Virginia


In [ ]:
def reshape_seq(seq, m ,n):
    
    r_seq=np.reshape(seq,(m,n))
    
    return r_seq

re_seqList=[]

for index, row in padded_final_data.iterrows():
    seq_ar=reshape_seq(row["padded_enc_seq"],n,max_len//n)
    re_seqList.append(seq_ar)

In [ ]:
X = np.asarray(re_seqList)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))
X.shape

(4484, 6, 4987, 1)

In [ ]:
Y = padded_final_data['Geo_Location'].values
Y.shape

(4484,)

In [ ]:
loc_classes = list(Y)
loc_classes = np.array(loc_classes) 
loc_classes = list(np.unique(loc_classes))

In [ ]:
df = padded_final_data['Geo_Location'].apply(loc_classes.index)
Y = df.values
Y.shape

(4484,)

In [ ]:
!pip install keras-metrics

In [ ]:
acc_per_fold = []
loss_per_fold = []
cvscores=[]
num_folds = 3 # there will be 3 folds in it's entirety
kfold = KFold(n_splits=num_folds, shuffle=True)

In [ ]:
print(X.shape)

(4484, 6, 4987, 1)


In [ ]:
img_rows, img_cols = X.shape[1],X.shape[2]
Y = keras.utils.to_categorical(Y,8)

In [ ]:
model_best = Sequential()

In [ ]:
class CheckMetrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_accs = []

    def on_epoch_end(self, epoch, logs={}):
        X_val, Y_val = self.validation_data[0],self.validation_data[1]
                
        Y_val = np.argmax(Y_val,axis=1)
        
        Y_pred = self.model.predict(X_val)
        Y_pred = np.argmax(Y_pred,axis=1)

        _val_acc = accuracy_score(Y_val, Y_pred)

        self.val_accs.append(_val_acc)
        
        if _val_acc == max(self.val_accs):
            print("Validation Accuracy has improved. Saving Model.")
            self.model.save('modelCNNGeo.h5')
            model_best = self.model

        return

In [ ]:
keras_callbacks = CheckMetrics()

In [ ]:
fold_no = 1
num_classes = 8
for train,test in kfold.split(X,Y):
  model = Sequential()
  model.add(Conv2D(16, kernel_size=(3,3), activation='relu', input_shape=(img_rows,img_cols,1)))
  model.add(Conv2D(16, (3, 3),activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, (3,3), padding='same',activation='relu'))
  model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(16, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  # compilation of model
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
  print('------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  model.fit(X[train],Y[train], batch_size = 64, epochs = 200,verbose = 1, validation_data= (X[test],Y[test]), callbacks=[keras_callbacks])
  model_sel = load_model('modelCNNGeo.h5')
  Y_pred = model_sel.predict(X[test])
  y_pred = np.argmax(Y_pred, axis=1)
  print('Confusion matrix - Test')
  y_act = np.argmax(Y[test],axis=1)
  print(confusion_matrix(y_act,y_pred))
  print('Classification report')
  target_names= ['California','Connecticut','Florida','Masachusetts','Michigan','New York','Virginia','Washington']
  print(classification_report(y_act,y_pred,target_names=target_names))
  scores = model_sel.evaluate(X[test],Y[test],verbose=0)
  print(f'Score for fold {fold_no}: {model_sel.metrics_names[0]} of {scores[0]}; {model_sel.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])
  cvscores.append(scores[1]*100)
  fold_no = fold_no + 1

------------------------------------------------------------
Training for fold 1 ...
Train on 2989 samples, validate on 1495 samples
Epoch 1/200
2989/2989 [==============================] - 5s 2ms/step - loss: 1.8552 - accuracy: 0.3603 - val_loss: 1.6988 - val_accuracy: 0.4308
Validation Accuracy has improved. Saving Model.
Epoch 2/200
2989/2989 [==============================] - 4s 1ms/step - loss: 1.6602 - accuracy: 0.4500 - val_loss: 1.5381 - val_accuracy: 0.4970
Validation Accuracy has improved. Saving Model.
Epoch 3/200
2989/2989 [==============================] - 4s 1ms/step - loss: 1.5792 - accuracy: 0.4751 - val_loss: 1.3933 - val_accuracy: 0.5144
Validation Accuracy has improved. Saving Model.
Epoch 4/200
2989/2989 [==============================] - 4s 1ms/step - loss: 1.4848 - accuracy: 0.5115 - val_loss: 1.3559 - val_accuracy: 0.5452
Validation Accuracy has improved. Saving Model.
Epoch 5/200
2989/2989 [==============================] - 4s 1ms/step - loss: 1.4443 - accuracy:

In [ ]:
print('------------------------------------------------')
print('Score per fold')
for i in range(0, len(cvscores)):
  print('-----------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {cvscores[i]}%')
print('--------------------------------------------------')
print('Average score for all folds:')
print(f'> Accuracy: {np.mean(cvscores)}')
print(f'>Loss: {np.mean(loss_per_fold)}')
#print(model.summary())

------------------------------------------------
Score per fold
-----------------------------------------------------
> Fold 1 - Loss: 1.3297190999107615 - Accuracy: 80.73578476905823%
-----------------------------------------------------
> Fold 2 - Loss: 1.395339061588946 - Accuracy: 80.13377785682678%
-----------------------------------------------------
> Fold 3 - Loss: 1.2593145533358876 - Accuracy: 76.03748440742493%
--------------------------------------------------
Average score for all folds:
> Accuracy: 78.96901567776997
>Loss: 1.3281242382785317


In [ ]:
best_model = load_model('modelCNNGeo.h5')
Y_pred = best_model.predict(X[test])
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion matrix - Test')
y_act = np.argmax(Y[test],axis=1)
print(confusion_matrix(y_act,y_pred))
print('Classification report')
target_names= ['California','Connecticut','Florida','Masachusetts','Michigan','New York','Virginia','Washington']
print(classification_report(y_act,y_pred,target_names=target_names))

Confusion matrix - Test
[[174   0   1  11  19   3  25  19]
 [  6   0   0   3  18   0   0   9]
 [  1   0  42   0   2   2   1   2]
 [ 15   0   0  68   0   8   2  51]
 [ 33   0   0   5 118   0   0   1]
 [  9   1   0   5   1  74   7   1]
 [ 35   0   0   4   2   1  63  12]
 [ 14   0   0  20   1   2   6 597]]
Classification report
              precision    recall  f1-score   support

  California       0.61      0.69      0.65       252
 Connecticut       0.00      0.00      0.00        36
     Florida       0.98      0.84      0.90        50
Masachusetts       0.59      0.47      0.52       144
    Michigan       0.73      0.75      0.74       157
    New York       0.82      0.76      0.79        98
    Virginia       0.61      0.54      0.57       117
  Washington       0.86      0.93      0.90       640

    accuracy                           0.76      1494
   macro avg       0.65      0.62      0.63      1494
weighted avg       0.74      0.76      0.75      1494

